In [ ]:
import os
import pickle
import math
import numpy as np
import imageio
import matplotlib.pyplot as plt

from implicitpleth.utils.algorithmic import CHROME_DEHAAN, POS_WANG, detrend_and_filter, ICA_POH
from implicitpleth.utils.eval import eval_performance, eval_clinical_performance, eval_performance_bias
from implicitpleth.utils.errors import getErrors
from implicitpleth.utils.utils import prpsd2

# Residual Pleth + Appearance 

In [ ]:
PPG_ROOT = './rgb_files/'
NPY_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes'
APP_ROOT = '/home/pradyumnachari/Desktop/AppNpy'
filetag = 'residual_'
filetag_app = 'appearance_'
session_names = os.listdir(PPG_ROOT)

error_green_list = []
error_chrom_list = []
error_pos_list = []
error_ica_list = []
error_green_proc_list = []
green_list = []
chrom_list = []
pos_list = []
ica_list = []
gt_list = []
green_proc_list = []
for _trial in session_names:
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[25:]

    print(os.path.join(NPY_ROOT, f'{filetag}0_{_trial}.npy'))
    pleth_1 = np.load(os.path.join(NPY_ROOT, f'{filetag}0_{_trial}.npy')).mean(1).mean(1)
    pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
    app_1 = np.load(os.path.join(APP_ROOT, f'{filetag_app}0_{_trial}.npy')).mean(1).mean(1)
    pleth_1 = pleth_1 + app_1*0.1

    pleth_2 = np.load(os.path.join(NPY_ROOT, f'{filetag}300_{_trial}.npy')).mean(1).mean(1)
    pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
    app_2 = np.load(os.path.join(APP_ROOT, f'{filetag_app}300_{_trial}.npy')).mean(1).mean(1)
    pleth_2 = pleth_2 + app_2*0.1

    pleth_3 = np.load(os.path.join(NPY_ROOT, f'{filetag}600_{_trial}.npy')).mean(1).mean(1)
    pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
    app_3 = np.load(os.path.join(APP_ROOT, f'{filetag_app}600_{_trial}.npy')).mean(1).mean(1)
    pleth_3 = pleth_3 + app_3*0.1

    pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
    temp_green = []
    temp_chrom = []
    temp_pos = []
    temp_ica = []
    temp_gt = []
    temp_proc = []
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'GT: {gt_hr}')
        temp_gt.append(gt_hr)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        green_proc = detrend_and_filter(pleth[:,1])
        green_proc_hr = prpsd2(green_proc-np.mean(green_proc), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Proc Est: {green_proc_hr}    ;   Green Proc Error: {np.abs(green_proc_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')

        temp_green.append(green_hr)
        temp_chrom.append(chrome_dehaan_hr)
        temp_pos.append(pos_wang_hr)
        temp_ica.append(ica_hr)
        temp_proc.append(green_proc_hr)
        print('*'*100)
    hr_gt_windowed = np.array(temp_gt)

    hr_est_windowed = np.array([temp_green])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_list.append(MAE)

    
    hr_est_windowed = np.array([temp_pos])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pos_list.append(MAE)
    
    hr_est_windowed = np.array([temp_chrom])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_chrom_list.append(MAE)
    
    hr_est_windowed = np.array([temp_ica])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_ica_list.append(MAE)

    hr_est_windowed = np.array([temp_proc])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_proc_list.append(MAE)
    
    green_list.append(temp_green)
    chrom_list.append(temp_chrom)
    pos_list.append(temp_pos)
    ica_list.append(temp_ica)
    green_proc_list.append(temp_proc)
    
    gt_list.append(temp_gt)
    print('-'*100)

In [ ]:
print(f'green_error_list: {np.mean(error_green_list)}')
print(f'proc_error_list: {np.mean(error_green_proc_list)}')
print(f'pos_error_list: {np.mean(error_pos_list)}')
print(f'chrom_error_list: {np.mean(error_chrom_list)}')
print(f'ica_error_list: {np.mean(error_ica_list)}')


In [ ]:
fitz_labels_path='./fitzpatrick_labels.pkl'
print(100*"-")
print('!!   !!  Green  !!  !!')
eval_clinical_performance(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  proc  !!  !!')
eval_clinical_performance(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pos  !!  !!')
eval_clinical_performance(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  chrom  !!  !!')
eval_clinical_performance(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  ica  !!  !!')
eval_clinical_performance(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

# Algorithmic on Residual

In [ ]:
PPG_ROOT = './rgb_files/'
NPY_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/'
_trial = 'v_63_2'

session_names = os.listdir(PPG_ROOT)

error_green_list = []
error_chrom_list = []
error_pos_list = []
error_ica_list = []
error_green_proc_list = []
green_list = []
chrom_list = []
pos_list = []
ica_list = []
gt_list = []
green_proc_list = []
for _trial in session_names:
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[25:]

    print(os.path.join(NPY_ROOT, f'residual_0_{_trial}.npy'))
    pleth_1 = np.load(os.path.join(NPY_ROOT, f'residual_0_{_trial}.npy')).mean(1).mean(1)
    pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
    pleth_2 = np.load(os.path.join(NPY_ROOT, f'residual_300_{_trial}.npy')).mean(1).mean(1)
    pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
    pleth_3 = np.load(os.path.join(NPY_ROOT, f'residual_600_{_trial}.npy')).mean(1).mean(1)
    pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
    pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
    temp_green = []
    temp_chrom = []
    temp_pos = []
    temp_ica = []
    temp_gt = []
    temp_proc = []
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'GT: {gt_hr}')
        temp_gt.append(gt_hr)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        green_proc = detrend_and_filter(pleth[:,1])
        green_proc_hr = prpsd2(green_proc-np.mean(green_proc), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Proc Est: {green_proc_hr}    ;   Green Proc Error: {np.abs(green_proc_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
        
        temp_green.append(green_hr)
        temp_chrom.append(chrome_dehaan_hr)
        temp_pos.append(pos_wang_hr)
        temp_ica.append(ica_hr)
        temp_proc.append(green_proc_hr)
        print('*'*100)
    hr_gt_windowed = np.array(temp_gt)

    hr_est_windowed = np.array([temp_green])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_list.append(MAE)
    
    hr_est_windowed = np.array([temp_pos])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pos_list.append(MAE)
    
    hr_est_windowed = np.array([temp_chrom])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_chrom_list.append(MAE)
    
    hr_est_windowed = np.array([temp_ica])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_ica_list.append(MAE)

    hr_est_windowed = np.array([temp_proc])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_proc_list.append(MAE)
    
    green_list.append(temp_green)
    chrom_list.append(temp_chrom)
    pos_list.append(temp_pos)
    ica_list.append(temp_ica)
    green_proc_list.append(temp_proc)
    
    gt_list.append(temp_gt)

    # Errors

    print('-'*100)

In [ ]:
print(f'green_error_list: {np.mean(error_green_list)}')
print(f'proc_error_list: {np.mean(error_green_proc_list)}')
print(f'pos_error_list: {np.mean(error_pos_list)}')
print(f'chrom_error_list: {np.mean(error_chrom_list)}')
print(f'ica_error_list: {np.mean(error_ica_list)}')

In [ ]:
fitz_labels_path='./fitzpatrick_labels.pkl'
print(100*"-")
print('!!   !!  Green  !!  !!')
eval_clinical_performance(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  proc  !!  !!')
eval_clinical_performance(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pos  !!  !!')
eval_clinical_performance(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  chrom  !!  !!')
eval_clinical_performance(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  ica  !!  !!')
eval_clinical_performance(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

# Mask and Algorithmic on Residual + Appearance

In [ ]:
MASK_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/FinalMasks/avgMasks/'
PPG_ROOT = './rgb_files/'
NPY_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/'
APP_ROOT = '/home/pradyumnachari/Desktop/AppNpy'
filetag = 'residual_'
filetag_app = 'appearance_'
session_names = os.listdir(PPG_ROOT)

error_green_list = []
error_chrom_list = []
error_pos_list = []
error_ica_list = []
error_green_proc_list = []
green_list = []
chrom_list = []
pos_list = []
ica_list = []
gt_list = []
green_proc_list = []
for _trial in session_names:
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[25:]

    print(os.path.join(NPY_ROOT, f'{filetag}0_{_trial}.npy'))
    pleth_1 = np.load(os.path.join(NPY_ROOT, f'{filetag}0_{_trial}.npy'))
    mask_1 = np.load(os.path.join(MASK_ROOT, f'{_trial}_0.npy'))[np.newaxis,...,np.newaxis]
    # Weigh the pleth with the mask
    pleth_1 = (pleth_1*mask_1).sum(1).sum(1) / mask_1.sum(1).sum(1)
    pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
    # Weigh the appearance with the mask
    app_1 = np.load(os.path.join(APP_ROOT, f'{filetag_app}0_{_trial}.npy'))
    app_1 = (app_1*mask_1).sum(1).sum(1) / mask_1.sum(1).sum(1)
    pleth_1 = pleth_1 + app_1*0.1
    
    pleth_2 = np.load(os.path.join(NPY_ROOT, f'{filetag}300_{_trial}.npy'))
    mask_2 = np.load(os.path.join(MASK_ROOT, f'{_trial}_1.npy'))[np.newaxis,...,np.newaxis]
    # Weigh the pleth with the mask
    pleth_2 = (pleth_2*mask_2).sum(1).sum(1) / mask_2.sum(1).sum(1)
    pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
    # Weigh the appearance with the mask
    app_2 = np.load(os.path.join(APP_ROOT, f'{filetag_app}300_{_trial}.npy'))
    app_2 = (app_2*mask_2).sum(1).sum(1) / mask_2.sum(1).sum(1)
    pleth_2 = pleth_2 + app_2*0.1
    
    pleth_3 = np.load(os.path.join(NPY_ROOT, f'{filetag}600_{_trial}.npy'))
    mask_3 = np.load(os.path.join(MASK_ROOT, f'{_trial}_2.npy'))[np.newaxis,...,np.newaxis]
    # Weigh the pleth with the mask
    pleth_3 = (pleth_3*mask_3).sum(1).sum(1) / mask_3.sum(1).sum(1)
    pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
    # Weigh the appearance with the mask
    app_3 = np.load(os.path.join(APP_ROOT, f'{filetag_app}600_{_trial}.npy'))
    app_3 = (app_3*mask_3).sum(1).sum(1) / mask_3.sum(1).sum(1)
    pleth_3 = pleth_3 + app_3*0.1
    
    pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
    temp_green = []
    temp_chrom = []
    temp_pos = []
    temp_ica = []
    temp_gt = []
    temp_proc = []
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'GT: {gt_hr}')
        temp_gt.append(gt_hr)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        green_proc = detrend_and_filter(pleth[:,1])
        green_proc_hr = prpsd2(green_proc-np.mean(green_proc), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Proc Est: {green_proc_hr}    ;   Green Proc Error: {np.abs(green_proc_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')

        temp_green.append(green_hr)
        temp_chrom.append(chrome_dehaan_hr)
        temp_pos.append(pos_wang_hr)
        temp_ica.append(ica_hr)
        temp_proc.append(green_proc_hr)
        print('*'*100)
    hr_gt_windowed = np.array(temp_gt)

    hr_est_windowed = np.array([temp_green])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_list.append(MAE)

    
    hr_est_windowed = np.array([temp_pos])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pos_list.append(MAE)
    
    hr_est_windowed = np.array([temp_chrom])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_chrom_list.append(MAE)
    
    hr_est_windowed = np.array([temp_ica])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_ica_list.append(MAE)

    hr_est_windowed = np.array([temp_proc])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_proc_list.append(MAE)
    
    green_list.append(temp_green)
    chrom_list.append(temp_chrom)
    pos_list.append(temp_pos)
    ica_list.append(temp_ica)
    green_proc_list.append(temp_proc)
    
    gt_list.append(temp_gt)
    print('-'*100)

In [ ]:
print(f'green_error_list: {np.mean(error_green_list)}')
print(f'proc_error_list: {np.mean(error_green_proc_list)}')
print(f'pos_error_list: {np.mean(error_pos_list)}')
print(f'chrom_error_list: {np.mean(error_chrom_list)}')
print(f'ica_error_list: {np.mean(error_ica_list)}')


In [ ]:
fitz_labels_path='./fitzpatrick_labels.pkl'
print(100*"-")
print('!!   !!  Green  !!  !!')
eval_clinical_performance(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  proc  !!  !!')
eval_clinical_performance(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pos  !!  !!')
eval_clinical_performance(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  chrom  !!  !!')
eval_clinical_performance(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  ica  !!  !!')
eval_clinical_performance(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

# Mask and Algorithmic on Residual

In [ ]:
MASK_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/FinalMasks/avgMasks/'
PPG_ROOT = './rgb_files/'
NPY_ROOT = '/home/pradyumnachari/Documents/ImplicitPPG/ResidualData/NpyRes/'
filetag = 'residual_'
session_names = os.listdir(PPG_ROOT)

error_green_list = []
error_chrom_list = []
error_pos_list = []
error_ica_list = []
error_green_proc_list = []
green_list = []
chrom_list = []
pos_list = []
ica_list = []
gt_list = []
green_proc_list = []
for _trial in session_names:
    print(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(PPG_ROOT,_trial,"rgbd_ppg.npy"))[25:]

    print(os.path.join(NPY_ROOT, f'{filetag}0_{_trial}.npy'))
    pleth_1 = np.load(os.path.join(NPY_ROOT, f'{filetag}0_{_trial}.npy'))
    mask_1 = np.load(os.path.join(MASK_ROOT, f'{_trial}_0.npy'))[np.newaxis,...,np.newaxis]
    pleth_1 = (pleth_1*mask_1).sum(1).sum(1) / mask_1.sum(1).sum(1)
    pleth_1 = pleth_1 - np.mean(pleth_1, axis=0, keepdims=True)
    
    pleth_2 = np.load(os.path.join(NPY_ROOT, f'{filetag}300_{_trial}.npy'))
    mask_2 = np.load(os.path.join(MASK_ROOT, f'{_trial}_1.npy'))[np.newaxis,...,np.newaxis]
    pleth_2 = (pleth_2*mask_2).sum(1).sum(1) / mask_2.sum(1).sum(1)
    pleth_2 = pleth_2 - np.mean(pleth_2, axis=0, keepdims=True)
    
    pleth_3 = np.load(os.path.join(NPY_ROOT, f'{filetag}600_{_trial}.npy'))
    mask_3 = np.load(os.path.join(MASK_ROOT, f'{_trial}_2.npy'))[np.newaxis,...,np.newaxis]
    pleth_3 = (pleth_3*mask_3).sum(1).sum(1) / mask_3.sum(1).sum(1)
    pleth_3 = pleth_3 - np.mean(pleth_3, axis=0, keepdims=True)
    
    pleth_full = np.concatenate((pleth_1, pleth_2, pleth_3), axis=0)
    temp_green = []
    temp_chrom = []
    temp_pos = []
    temp_ica = []
    temp_gt = []
    temp_proc = []
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'GT: {gt_hr}')
        temp_gt.append(gt_hr)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        green_proc = detrend_and_filter(pleth[:,1])
        green_proc_hr = prpsd2(green_proc-np.mean(green_proc), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Proc Est: {green_proc_hr}    ;   Green Proc Error: {np.abs(green_proc_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')

        temp_green.append(green_hr)
        temp_chrom.append(chrome_dehaan_hr)
        temp_pos.append(pos_wang_hr)
        temp_ica.append(ica_hr)
        temp_proc.append(green_proc_hr)
        print('*'*100)
    hr_gt_windowed = np.array(temp_gt)

    hr_est_windowed = np.array([temp_green])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_list.append(MAE)

    
    hr_est_windowed = np.array([temp_pos])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pos_list.append(MAE)
    
    hr_est_windowed = np.array([temp_chrom])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_chrom_list.append(MAE)
    
    hr_est_windowed = np.array([temp_ica])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_ica_list.append(MAE)

    hr_est_windowed = np.array([temp_proc])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_proc_list.append(MAE)
    
    green_list.append(temp_green)
    chrom_list.append(temp_chrom)
    pos_list.append(temp_pos)
    ica_list.append(temp_ica)
    green_proc_list.append(temp_proc)
    
    gt_list.append(temp_gt)
    print('-'*100)

In [ ]:
print(f'green_error_list: {np.mean(error_green_list)}')
print(f'proc_error_list: {np.mean(error_green_proc_list)}')
print(f'pos_error_list: {np.mean(error_pos_list)}')
print(f'chrom_error_list: {np.mean(error_chrom_list)}')
print(f'ica_error_list: {np.mean(error_ica_list)}')


In [ ]:
fitz_labels_path='./fitzpatrick_labels.pkl'
print(100*"-")
print('!!   !!  Green  !!  !!')
eval_clinical_performance(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  proc  !!  !!')
eval_clinical_performance(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pos  !!  !!')
eval_clinical_performance(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  chrom  !!  !!')
eval_clinical_performance(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  ica  !!  !!')
eval_clinical_performance(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

# Baselines

In [ ]:
ROOT = './rgb_files/'

session_names = os.listdir(ROOT)

error_green_list = []
error_chrom_list = []
error_pos_list = []
error_ica_list = []
error_green_proc_list = []
green_list = []
chrom_list = []
pos_list = []
ica_list = []
gt_list = []
green_proc_list = []
for _trial in session_names:
    print(os.path.join(ROOT,_trial,"rgbd_ppg.npy"))
    gt_ppg_full = np.load(os.path.join(ROOT,_trial,"rgbd_ppg.npy"))[25:]

    pleth_full  = []
    for idx in range(900):
        pleth_full.append(imageio.v2.imread(os.path.join(ROOT, _trial, f'rgbd_rgb_{idx}.png')))
    pleth_full = np.array(pleth_full).mean(1).mean(1) / 255
    
    temp_green = []
    temp_chrom = []
    temp_pos = []
    temp_ica = []
    temp_gt = []
    temp_proc = []
    for i in [0,128,256,384,512]:
        gt_ppg = gt_ppg_full[i:i+300]
        gt_hr = prpsd2(gt_ppg-np.mean(gt_ppg), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'GT: {gt_hr}')
        temp_gt.append(gt_hr)
        
        pleth = pleth_full[i:i+300]
        green = pleth[:,1]
        green_hr = prpsd2(green-np.mean(green), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Est: {green_hr}    ;   Green Error: {np.abs(green_hr - gt_hr)}')
        green_proc = detrend_and_filter(pleth[:,1])
        green_proc_hr = prpsd2(green_proc-np.mean(green_proc), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Green Proc Est: {green_proc_hr}    ;   Green Proc Error: {np.abs(green_proc_hr - gt_hr)}')
        chrome_dehaan = CHROME_DEHAAN(pleth)
        chrome_dehaan_hr = prpsd2(chrome_dehaan-np.mean(chrome_dehaan), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'Chrome Est: {chrome_dehaan_hr}    ;   Chrome Error: {np.abs(chrome_dehaan_hr - gt_hr)}')
        pos_wang = POS_WANG(pleth)
        pos_wang_hr = prpsd2(pos_wang-np.mean(pos_wang), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'POS Est: {pos_wang_hr}    ;   POS Error: {np.abs(pos_wang_hr - gt_hr)}')
        ica_poh = ICA_POH(pleth)
        ica_hr = prpsd2(ica_poh-np.mean(ica_poh), 30, 45, 150, BUTTER_ORDER=6, RECT=True)
        print(f'ICA Est: {ica_hr}    ;   ICA Error: {np.abs(ica_hr - gt_hr)}')
        
        temp_green.append(green_hr)
        temp_chrom.append(chrome_dehaan_hr)
        temp_pos.append(pos_wang_hr)
        temp_ica.append(ica_hr)
        temp_proc.append(green_proc_hr)
        print('*'*100)
    hr_gt_windowed = np.array(temp_gt)

    hr_est_windowed = np.array([temp_green])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_list.append(MAE)
    
    hr_est_windowed = np.array([temp_pos])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_pos_list.append(MAE)
    
    hr_est_windowed = np.array([temp_chrom])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_chrom_list.append(MAE)
    
    hr_est_windowed = np.array([temp_ica])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_ica_list.append(MAE)

    hr_est_windowed = np.array([temp_proc])
    RMSE, MAE, MAX, PCC = getErrors(hr_est_windowed, hr_gt_windowed)
    error_green_proc_list.append(MAE)
    
    green_list.append(temp_green)
    chrom_list.append(temp_chrom)
    pos_list.append(temp_pos)
    ica_list.append(temp_ica)
    green_proc_list.append(temp_proc)
    
    gt_list.append(temp_gt)

    # Errors

    print('-'*100)

In [ ]:
print(f'green_error_list: {np.mean(error_green_list)}')
print(f'proc_error_list: {np.mean(error_green_proc_list)}')
print(f'pos_error_list: {np.mean(error_pos_list)}')
print(f'chrom_error_list: {np.mean(error_chrom_list)}')
print(f'ica_error_list: {np.mean(error_ica_list)}')

In [ ]:
fitz_labels_path='./fitzpatrick_labels.pkl'
print(100*"-")
print('!!   !!  Green  !!  !!')
eval_clinical_performance(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  proc  !!  !!')
eval_clinical_performance(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(green_proc_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  pos  !!  !!')
eval_clinical_performance(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(pos_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  chrom  !!  !!')
eval_clinical_performance(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(chrom_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")

print('!!   !!  ica  !!  !!')
eval_clinical_performance(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(50*"*")
eval_performance_bias(hr_est=np.array(ica_list), hr_gt=np.array(gt_list), fitz_labels_path=fitz_labels_path, session_names=session_names)
print(100*"-")